In [1]:
import os
import os.path as osp
import numpy as np
import math
import cv2
import argparse

METER_SHAPE = 512
CIRCLE_CENTER = [256, 256]
CIRCLE_RADIUS = 250
PI = 3.1415926536
LINE_HEIGHT = 120
LINE_WIDTH = 1570  # 任意长度，之后可改成周长
TYPE_THRESHOLD = 33
METER_CONFIG = [{
    'scale_value': 1.6 / 32.0,
    'range': 1.6
}, {
    'scale_value': 25 / 50.0,
    'range': 25
}]

def read_process(label_maps):
    label_maps = use_erode_image(label_maps)
    # Convert the circular meter into rectangular meter
    line_images = creat_line_image(label_maps)
    cv2.imshow('line_image',line_images)
    # print(line_images.shape)
    # Convert the 2d meter into 1d meter
    scale_data, pointer_data = convert_1d_data(line_images)
    # Fliter scale data whose value is lower than the mean value
    scale_mean_filtration(scale_data)
    # Get scale_num, scales and ratio of meters
    result = get_meter_reader(scale_data, pointer_data)
    # fix scale
    result = fix_initial_scale(result)
    return result

# 腐蚀
def use_erode_image(meter_image, erode_kernel=3):
    kernel = np.ones((erode_kernel, erode_kernel), np.uint8)
    erode_image = cv2.erode(meter_image, kernel)
    return erode_image

# 转矩形
def creat_line_image( meter_image):
    line_image = np.zeros((LINE_HEIGHT, LINE_WIDTH), dtype=np.uint8)
    for row in range(LINE_HEIGHT):
        for col in range(LINE_WIDTH):
            theta = PI * 2 / LINE_WIDTH * (col + 1)
            rho = CIRCLE_RADIUS - row - 1
            h = int(CIRCLE_CENTER[1] + rho * math.cos(theta) + 0.5) # 这里的加减决定起始方向
            w = int(CIRCLE_CENTER[0] - rho * math.sin(theta) + 0.5)
            line_image[row, col] = meter_image[h, w]
            # if col <40:  # TODO :为什么line_image不是从theta = 0开始
            #     cv2.circle(meter_image, (x,y), 1, 255, 4)
            #     cv2.circle(line_image, (row, col), 1, 155, 4)
                # print(x, y, row, col, theta, meter_image[x, y])
    return line_image

# 转1D
def convert_1d_data( meter_image):
    scale_data = np.zeros((LINE_WIDTH), dtype=np.uint8)
    pointer_data = np.zeros((LINE_WIDTH), dtype=np.uint8)
    for col in range(LINE_WIDTH):
        for row in range(LINE_HEIGHT):
            if meter_image[row, col] == 38:
                pointer_data[col] += 1
            elif meter_image[row, col] == 75:
                scale_data[col] += 1
    return scale_data, pointer_data


def scale_mean_filtration(scale_data):
    data_w_id = np.where(scale_data>0)
    mean_data = np.mean(scale_data)
    for col in range(LINE_WIDTH):
        if scale_data[col] < mean_data:
            scale_data[col] = 0

def get_meter_reader(scale_data, pointer_data):
    scale_flag = False
    pointer_flag = False
    one_scale_start = 0
    one_scale_end = 0
    one_pointer_start = 0
    one_pointer_end = 0
    scale_location = list()
    scale_width = list()
    scale_width_mean = 0  # 刻度平均宽
    scale_width_first = 0  # 第一个刻度宽
    scale_range = list()  # 刻度之间的距离
    scale_range_median = 0  # 刻度间隔中位数
    pointer_location = 0
    max_pointer_location = 0
    for i in range(LINE_WIDTH - 1):
        if scale_data[i] > 0 and scale_data[i + 1] > 0:
            if scale_flag == False:
                one_scale_start = i
                scale_flag = True
        if scale_flag:
            if scale_data[i] == 0 and scale_data[i + 1] == 0:
                one_scale_end = i - 1
                one_scale_location = (one_scale_start + one_scale_end) / 2
                scale_location.append(one_scale_location)
                scale_width.append(one_scale_end - one_scale_start)
                if len(scale_location) > 1:  # 不计算第一个刻度的长度
                    scale_width_mean = (scale_width_mean + (one_scale_end - one_scale_start))/2
                else:
                    scale_width_first = one_scale_end - one_scale_start
                one_scale_start = 0
                one_scale_end = 0
                scale_flag = False
        if pointer_data[i] > 0 and pointer_data[i + 1] > 0:
            if pointer_flag == False:
                one_pointer_start = i
                pointer_flag = True
        if pointer_flag:
            if pointer_data[i] == 0 and pointer_data[i + 1] == 0:
                one_pointer_end = i - 1
                # 去除杂点，选取最大指针距离的
                if (one_pointer_end - one_pointer_start) > max_pointer_location:
                    pointer_location = (one_pointer_start + one_pointer_end) / 2
                    max_pointer_location = one_pointer_end - one_pointer_start
                one_pointer_start = 0
                one_pointer_end = 0
                pointer_flag = False

    for i in range(len(scale_location)-1):  # 求刻度间隔
        scale_range.append(scale_location[i+1] - scale_location[i])
    scale_range_median = np.median(scale_range)
    # print(scale_width)
    for i in range(len(scale_width)-1):  # 寻找合并的刻度,默认scale_range插入
        if scale_width[i+1] > scale_width_mean*1.9:
            merge_num = int(scale_width[i+1]/scale_width_mean)
            if merge_num==2:  # 最多修复2个刻度合并情况，不然就用角度计算
                scale_location.pop(i+1)
                for j in range(merge_num):
                    scale_location.insert(i+1+j, scale_location[i]+scale_range_median*(j+1))

    scale_num = len(scale_location)
    scales = -1
    ratio = -1
    ratio_add = -1
    if scale_num > 0:
        for i in range(scale_num - 1):
            if scale_location[i] <= pointer_location and pointer_location < scale_location[i + 1]:
                scales = i + (pointer_location - scale_location[i]) / (scale_location[i + 1] - scale_location[i] + 1e-05) 
                break
        ratio = (pointer_location - scale_location[0]) / (scale_location[scale_num - 1] - scale_location[0] + 1e-05)
        # 模拟计算首刻度有问题情况下的角度，用不用之后判断
        ratio_add = (pointer_location - scale_location[1] + scale_range_median*2) / (
                     scale_location[scale_num - 1] - scale_location[1] + scale_range_median*2 + 1e-05)
    
    result = {'scale_num': scale_num, 'scales': scales, 'ratio': ratio,
              'scale_width_mean': scale_width_mean,
              'scale_width_first': scale_width_first,
              'scale_range_median': scale_range_median,
              'ratio_add': ratio_add}
    return result

def fix_initial_scale(result):
    if result['scale_width_first'] > result['scale_width_mean']*1.5:  # 如果第一个刻度过宽，说明第一二个刻度合并
        print('add initial scale.')
        result['scale_num'] = result['scale_num'] + 1
        result['scales'] = result['scales'] + 1
        result['ratio'] = result['ratio_add']
    elif (result['scale_num'] - TYPE_THRESHOLD) == 1:  # 没有过宽，但确实合并了
        result['scale_num'] = result['scale_num'] + 1
        result['scales'] = result['scales'] + 1
        result['ratio'] = result['ratio_add']
        return result
    if (TYPE_THRESHOLD%2) == 0:  # 如果缺少第一个刻度
        print('lach 1 scale')
        result['scales'] = result['scales'] + 1
        result['ratio'] = result['ratio_add']
    return result

In [2]:
# 测试检测指针度数
import glob
import random


if __name__ == '__main__':
    meter_img = cv2.imread('E:\\sc\\image_data\\meter\\meter_seg\\images\\test_true\\1.jpg')
    src_img = cv2.imread('E:\\sc\\image_data\\meter\\meter_seg\\images\\test_true\\1_mask.png', 0)
    # print(src_img.shape)

    src_img = cv2.resize(src_img, (METER_SHAPE, METER_SHAPE))
    result = read_process(src_img)
    print(result)

    if result['scale_num'] == TYPE_THRESHOLD:  # 如果刻度相等
        print('all scales are checked')
        value = result['scales'] * METER_CONFIG[0]['scale_value']
    else:  # 如果不等，说明缺少。
        print('lack of scale,use ratio')
        value = result['ratio'] * METER_CONFIG[0]['range']
    print("-- Meter result: {} --\n".format(value))

    cv2.imshow('meter_img',meter_img)
    cv2.imshow('src_img',src_img)
    cv2.waitKey(0)

add initial scale.
{'scale_num': 51, 'scales': 12.81818144628116, 'ratio': 0.24889282330475798, 'scale_width_mean': 5.018435012120918, 'scale_width_first': 12, 'scale_range_median': 22.5, 'ratio_add': 0.24889282330475798}
lack of scale,use ratio
-- Meter result: 0.39822851728761277 --



In [15]:
from PIL import Image, ImageOps
import cv2
import time
import numpy as np



st1 = time.time()
im1 = Image.open(r'/home/y/sc_dev/dilun/image_data/meter/meter_seg/images/test/380.jpg').convert('RGB')  # 读取图形pil更快 RGB
# im2 = ImageOps.expand(im1, border=(0, 0, 20, 40), fill = 150)
# im1 = im1.resize((513, 513))
# img1 = cv2.cvtColor(np.asarray(im1), cv2.COLOR_RGB2BGR)  # PIL-CV2
end1 = time.time()

st2 = time.time()
img2 = cv2.imread(r'/home/y/sc_dev/dilun/image_data/meter/meter_seg/images/test/380.jpg')  # BGR
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
# img2 = cv2.resize(img2, (513, 513))  # 处理图形，cv2更快
# im2 = Image.fromarray(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))  # cv2-pil
end2 = time.time()

print(end1 - st1, end2 - st2)
# print(np.array(im1).shape, img2.shape)

# im2.show()
# cv2.imshow('2', img2)
# cv2.waitKey(0)

0.008672952651977539 0.006470203399658203
